In [1]:
import os

In [2]:
os.getcwd()

'c:\\Users\\Archana\\Desktop\\test\\assignmen_solution\\research'

In [3]:
os.chdir('../')

In [32]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [33]:
from src.ml_proj.utils.common import read_yaml, create_directories
from src.ml_proj.utils.common import save_json
from src.ml_proj.constants import *

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config1 = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        config2 = self.config.mlflow_info

        os.environ['MLFLOW_TRACKING_URI'] = config2.tracking_uri
        os.environ['MLFLOW_TRACKING_USERNAME'] = config2.tracking_username
        os.environ['MLFLOW_TRACKING_PASSWORD'] = config2.tracking_password
        create_directories([config1.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config1.root_dir,
            test_data_path=config1.test_data_path,
            model_path = config1.model_path,
            all_params=params,
            metric_file_name = config1.metric_file_name,
            target_column = schema.name,
            mlflow_uri= config2.tracking_uri,
           
        )

        return model_evaluation_config

In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
import joblib

In [36]:
import json
from urllib.parse import urlparse

In [37]:
%pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [38]:
import mlflow
import mlflow.sklearn

In [39]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_experiment("First Experiment")
        
        mlflow.set_tracking_uri(uri=self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            accuracy = accuracy_score(test_y, predicted_qualities)
            save_json(path=Path(self.config.metric_file_name), data=scores)
            

            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("accuracy",accuracy)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="LogisticRegressionModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    

In [40]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
    
except Exception as e:
    raise e

[2025-03-27 14:34:38,768: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-27 14:34:38,776: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 14:34:38,783: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-27 14:34:38,785: INFO: common: created directory at: artifacts]
[2025-03-27 14:34:38,786: INFO: common: created directory at: artifacts/model_evaluation]


2025/03/27 14:34:39 INFO mlflow.tracking.fluent: Experiment with name 'First Experiment' does not exist. Creating a new experiment.


[2025-03-27 14:34:41,121: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/03/27 14:34:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'LogisticRegressionModel'.
2025/03/27 14:35:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegressionModel, version 1
Created version '1' of model 'LogisticRegressionModel'.


🏃 View run resilient-crab-422 at: https://dagshub.com/archanalonkar/assignmen_solution.mlflow/#/experiments/0/runs/d34d399cafd4426e9e577672794b4827
🧪 View experiment at: https://dagshub.com/archanalonkar/assignmen_solution.mlflow/#/experiments/0
